In [1]:
URI = "gs://tfl-cycle-1413//app/airflow/data/20230612.parquet"


In [2]:
import pandas as pd
df = pd.read_parquet(URI)


In [3]:
df.head(2)


,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
0,131615942,2023-06-18 23:59,1106,"Rampayne Street, Pimlico",2023-06-19 00:03,1087,"Belgrave Road, Victoria",56222,CLASSIC,4m 16s,256371
1,131615943,2023-06-18 23:59,1204,"Queen's Gate (North), Kensington",2023-06-19 00:15,200166,"Southerton Road, Hammersmith",59682,CLASSIC,15m 23s,923968


In [1]:
from google.cloud import storage

def get_all_files_dates(bucket_name):
    storage_client = storage.Client.from_service_account_json('/home/d.bodin03/.gcp_keys/le-wagon-de-bootcamp.json')
    bucket = storage_client.get_bucket(bucket_name)

    iterator = bucket.list_blobs(prefix='bronze/')
    response = iterator._get_next_page_response()
    return [i['name'][7:15] for i in response['items']]
data = get_all_files_dates("tfl-cycle-1413")


In [2]:
keys = [d for d in get_all_files_dates("tfl-cycle-1413")]


['20150104',
 '20150118',
 '20150201',
 '20150215',
 '20150301',
 '20150316',
 '20150401',
 '20150417',
 '20150503',
 '20150517',
 '20150531',
 '20150613',
 '20150628',
 '20150712',
 '20150726',
 '20150808',
 '20150823',
 '20150906',
 '20150920',
 '20151004',
 '20151018',
 '20151101',
 '20151115',
 '20151128',
 '20151213',
 '20151225',
 '20160110',
 '20160124',
 '20160207',
 '20160221',
 '20160306',
 '20160401',
 '20160501',
 '20160518',
 '20160525',
 '20160601',
 '20160608',
 '20160615',
 '20160622',
 '20160629',
 '20160706',
 '20160713',
 '20160720',
 '20160727',
 '20160803',
 '20160810',
 '20160817',
 '20160824',
 '20160831',
 '20160907',
 '20160914',
 '20160921',
 '20160928',
 '20161005',
 '20161012',
 '20161019',
 '20161026',
 '20161102',
 '20161109',
 '20161116',
 '20161123',
 '20161130',
 '20161207',
 '20161214',
 '20161221',
 '20161228',
 '20170104',
 '20170111',
 '20170118',
 '20170125',
 '20170201',
 '20170208',
 '20170215',
 '20170222',
 '20170301',
 '20170308',
 '20170322',

In [21]:
!pwd


/home/d.bodin03/code/DanyBodin/tfl-cycle


In [4]:
df.columns


Index(['Number', 'Start date', 'Start station number', 'Start station',
       'End date', 'End station number', 'End station', 'Bike number',
       'Bike model', 'Total duration', 'Total duration (ms)'],
      dtype='object')

In [8]:
df['Total duration (ms)'] = df['Total duration (ms)'].map(lambda x : x*0.001)

df = df.rename(columns={"Total duration (ms)":"Total duration"})
df.head()


,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration
0,131615942,2023-06-18 23:59,1106,"Rampayne Street, Pimlico",2023-06-19 00:03,1087,"Belgrave Road, Victoria",56222,CLASSIC,4m 16s,2.563710e-07
1,131615943,2023-06-18 23:59,1204,"Queen's Gate (North), Kensington",2023-06-19 00:15,200166,"Southerton Road, Hammersmith",59682,CLASSIC,15m 23s,9.239680e-07
2,131615944,2023-06-19 00:00,22179,"Exhibition Road, Knightsbridge",2023-06-19 00:13,200120,"Grant Road East, Clapham Junction",55015,CLASSIC,13m 19s,7.992960e-07
3,131615939,2023-06-18 23:58,3504,"Moor Street, Soho",2023-06-19 00:55,1150,"Queensway, Kensington Gardens",41885,CLASSIC,56m 59s,3.419163e-06
4,131615940,2023-06-18 23:58,300082,"Bridge Avenue, Hammersmith",2023-06-19 00:04,200181,"Richmond Way, Shepherd's Bush",40935,CLASSIC,5m 27s,3.270400e-07
